In [ ]:
# see link
# https://github.com/Fraud-Detection-Handbook

package ='05-sklearn.tree'
name='Decision Tree'
tuningAndParameters='04-Random Oversampling'

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import datamanagement as dm
reload(dm)
import result as resultMd
reload(resultMd)

print('done')

In [ ]:
dfLearning, dfValidation =dm.getDataLearningAndValidation()

dfLearning.head()

In [ ]:
%%script false
# found rate = 6.5

from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
import numpy as np
from sklearn.metrics import f1_score

train_f1s=[]
test_f1s =[]
range= []

x_train0, x_test, y_train0, y_test = train_test_split(dfLearning[predictors], dfLearning[target], test_size = TEST_SIZE, 
                                                        stratify=dfLearning[target],
                                                        random_state = RANDOM_STATE)

ratio=dfLearning[target].value_counts()[1]/dfLearning[target].value_counts()[0]
print(ratio)

rates = np.arange(1.5,16,0.25)
for rate in rates:
    print("",rate,1/rate)
    oversample = RandomOverSampler(sampling_strategy=rate*ratio,random_state=42)
    x_train, y_train = oversample.fit_resample(x_train0, y_train0)


    print(x_train0.shape)
    print(x_train.shape)
    
    modelClf = DecisionTreeClassifier(random_state=42)
    parameters={'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 20}
    modelClf.set_params(**parameters)

    modelClf.fit(x_train, y_train)
    predsTrain = modelClf.predict(x_train)
    predsTest = modelClf.predict(x_test)

    train_f1=f1_score(y_train, predsTrain)
    print("f1 train {:.4f}".format(train_f1))
    
    test_f1=f1_score(y_test, predsTest)
    print("f1 test  {:.4f}".format(test_f1))
    
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(rate)
    print('-----------------------')

dm.plt_train_test(range, train_f1s, "train", test_f1s," test")

In [ ]:
%%script false
dm.plt_train_test(range, train_f1s, "train", test_f1s," test")

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0

predictors = dm.getPredictors(dfLearning)
target = dm.getTarget()

x_train0, x_test, y_train0, y_test = train_test_split(dfLearning[predictors], dfLearning[target], test_size = TEST_SIZE, 
                                                        stratify=dfLearning[target],
                                                        random_state = RANDOM_STATE)

ratio=dfLearning[target].value_counts()[1]/dfLearning[target].value_counts()[0]
print(ratio)
rate = 6.5
oversample = RandomOverSampler(sampling_strategy=rate*ratio,random_state=42)
x_train, y_train = oversample.fit_resample(x_train0, y_train0)

In [ ]:
%%script false

from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

#{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 7, 'min_samples_split': 32
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': randint(2,10),
    'min_samples_leaf': randint(2,10),
    'min_samples_split': randint(10,20)
}
modelClf = DecisionTreeClassifier(random_state=42)

random_search = RandomizedSearchCV(modelClf,dic_param, scoring='f1', verbose=10,cv=4).fit(x_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)



#{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 5, 'min_samples_split': 18}
#0.7229329651146276

In [ ]:
%%script false

from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV

#{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 7, 'min_samples_split': 32
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': [19,20,21,22,23],
    'min_samples_leaf': [3,4,5,6,7],
    'min_samples_split': [7,8,9,10,11]
}
modelClf = DecisionTreeClassifier(random_state=42)

random_search = GridSearchCV(modelClf,dic_param, scoring='f1', verbose=10,cv=4).fit(x_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)


#{'criterion': 'entropy', 'max_depth': 11, 'min_samples_leaf': 3, 'min_samples_split': 20}
#0.7752620821754512

#{'criterion': 'entropy', 'max_depth': 13, 'min_samples_leaf': 3, 'min_samples_split': 18}
#0.8271462961729861

#{'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 3, 'min_samples_split': 16}
#0.8677728162473197

#{'criterion': 'gini', 'max_depth': 17, 'min_samples_leaf': 3, 'min_samples_split': 14}
#0.8949600658094995

#{'criterion': 'gini', 'max_depth': 19, 'min_samples_leaf': 3, 'min_samples_split': 12}
#0.9149655214958644

#{'criterion': 'gini', 'max_depth': 21, 'min_samples_leaf': 3, 'min_samples_split': 11}
#0.9302945736714128

#{'criterion': 'gini', 'max_depth': 21, 'min_samples_leaf': 3, 'min_samples_split': 9}
#0.9312437870100525


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier

modelClf = DecisionTreeClassifier(random_state=42)
parameters={'criterion': 'gini', 'max_depth': 21, 'min_samples_leaf': 3, 'min_samples_split': 9}

modelClf.set_params(**parameters)
modelClf.fit(x_train, y_train)

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

dm.show_confusion_matrix(y_train, predsTrain,'Confusion matrix learning data')
print(f"f1 train {f1_score(y_train, predsTrain):.3f}")
dm.show_confusion_matrix(y_test, predsTest,'Confusion matrix test data')
print(f"f1 test {f1_score(y_test, predsTest):.3f}")

In [ ]:
predsValidation = modelClf.predict(dfValidation[predictors])
f1=f1_score(dfValidation[target], predsValidation)

dm.show_confusion_matrix(dfValidation[target], predsValidation,'Confusion matrix validation data')
print(f"f1 validation {f1:.3f}")
resultMd.update_performance_result(package, name, tuningAndParameters, f1)